In [1]:
import torch
from TorchDataUtils import *
from AGNewsDataset import *

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [9]:
df = pd.DataFrame(columns=['Title', 'Class Label'])
df = df.append({'Title': 'Riti', 'Class Label': 1}, ignore_index=True)
df2 = pd.DataFrame(columns=['Title', 'Class Label'])
df2 = df2.append(df)
df2

,Title,Class Label
0,Riti,1


# Manejo de datos en NLP

Todo esto está sacado de https://github.com/joosthub/PyTorchNLPBook, que es el github del libro que usan en cs224n. El capítulo 3 tiene un ejemplo "Classifying Sentiment of Restaurant Reviews" que es la fuente de toda esta información.

## Ciclo de trabajo

Para hacer un modelo de NLP, hay que hacer varias cosas:

1. Preprocesar el texto, de manera de obtener un .csv con todas las muestras de training y de test.

2. Definir un vocabulario. Para eso, se suele hacer un objeto a parte que contenga todos los handlers del vocabulario.

3. Definir un Vectorizer, que es un objeto que se usa para convertir las muestras del dataset en vectores que se puedan pasar al modelo.

4. Definir un objeto Dataset, que represente a las muestras en su conjunto. 

5. Definir un Dataloader que  maneje el muestreo aleatorio del dataset, la cantidad de muestras de validación, cargar las muestras a las gpu's y eso.

6. Recién en este paso, se puede hacer el modelo.

In [2]:
train_dataset = AGNewsDataset(root='./AG_NEWS/', train=True, use_test_tokens=True)
test_dataset = AGNewsDataset(root='./AG_NEWS/', train=False, use_test_tokens=True)

print("""Train Dataset:
Cantidad de muestras de entrenamiento: {}
Tamaño de cada muestra: {}
Categorías: ['World', 'Sports', 'Business', 'Sci/Tech']
""".format(len(train_dataset), len(train_dataset[0][0])))

print("""Test Dataset:
Cantidad de muestras de testeo: {}
Tamaño de cada muestra: {}
Categorías: ['World', 'Sports', 'Business', 'Sci/Tech']
""".format(len(test_dataset), len(test_dataset[0][0])))

print("Tamaño del vocabulario: {}".format(len(train_dataset.vocabulary)))
print("Se usan las palabras del train y del test")

train_dataloader, val_dataloader, test_dataloader = generate_data_batches(train_dataset, test_dataset,
                                                                         batch_size=128)

Train Dataset:
Cantidad de muestras de entrenamiento: 120000
Tamaño de cada muestra: 19
Categorías: ['World', 'Sports', 'Business', 'Sci/Tech']

Test Dataset:
Cantidad de muestras de testeo: 7600
Tamaño de cada muestra: 17
Categorías: ['World', 'Sports', 'Business', 'Sci/Tech']

Tamaño del vocabulario: 73916
Se usan las palabras del train y del test


In [3]:
import torch.nn as nn

class TextClassifier(nn.Module):
    
    def __init__(self, vocab_size, n_classes):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, n_classes)
        
    def forward(self, x):
        return self.emb(x).mean(dim=1)
    
    def loss(self, scores, target):
        lf = nn.CrossEntropyLoss()
        return lf(scores, target)
    
vocab_size = len(train_dataset.vocabulary)
n_classes = 4
model = TextClassifier(vocab_size, n_classes)

In [5]:
# Parámetros de las muestras:
data = {
    'use_gpu': True, # Trasladar o no las muestras a la GPU
    'input_dtype': torch.long, # Tipo de dato de las muestras de entrada
    'target_dtype': torch.long, # Tipo de dato de las muestras de salida
    'train_dataloader': train_dataloader, # Dataset de entrenamiento
    'val_dataloader': val_dataloader # Dataset de validación
}

# Parámetros de optimización:
epochs = 10 # Cantidad de epochs
sample_loss_every = 100 # Cantidad de iteraciones para calcular la cantidad de aciertos
learning_rate = 1e-4 # Tasa de aprendizaje
check_on_train = False # Queremos ver los resultados también en el train set

# Entrenamiento:
performance_history = SGDTrainModel(model, data, epochs, learning_rate, sample_loss_every, check_on_train)

Epoch: 0, Batch number: 0
Accuracy on validation dataset: 636/2400 (26.50%)

Epoch: 0, Batch number: 100
Accuracy on validation dataset: 636/2400 (26.50%)

Epoch: 0, Batch number: 200
Accuracy on validation dataset: 636/2400 (26.50%)

Epoch: 0, Batch number: 300
Accuracy on validation dataset: 636/2400 (26.50%)

Epoch: 0, Batch number: 400
Accuracy on validation dataset: 636/2400 (26.50%)

Epoch: 0, Batch number: 500
Accuracy on validation dataset: 637/2400 (26.54%)

Epoch: 0, Batch number: 600
Accuracy on validation dataset: 637/2400 (26.54%)

Epoch: 0, Batch number: 700
Accuracy on validation dataset: 637/2400 (26.54%)

Epoch: 0, Batch number: 800
Accuracy on validation dataset: 637/2400 (26.54%)

Epoch: 0, Batch number: 900
Accuracy on validation dataset: 637/2400 (26.54%)

Epoch: 1, Batch number: 81
Accuracy on validation dataset: 637/2400 (26.54%)

Epoch: 1, Batch number: 181
Accuracy on validation dataset: 637/2400 (26.54%)

Epoch: 1, Batch number: 281
Accuracy on validation data